# 과제물 자동 평가 시스템

In [1]:
import os
import sys
from dotenv import load_dotenv

# 환경 변수 로드
load_dotenv()

# 모듈 경로 추가
sys.path.append(os.path.abspath('../'))

# 모듈형 위젯 라이브러리 import
from widgets_core import (
    ChecklistCreationManager,
    AssignmentEvaluationManager,
    LlmCallManager
)

In [3]:
from openai import OpenAI
from pydantic import BaseModel

client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY")
)

class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[str]

response = client.responses.parse(
    model="gpt-4o-2024-08-06",
    input=[
        {"role": "system", "content": "Extract the event information."},
        {
            "role": "user",
            "content": "Alice and Bob are going to a science fair on Friday.",
        },
    ],
    text_format=CalendarEvent,
)

event = response.output_parsed

In [5]:
response

ParsedResponse[CalendarEvent](id='resp_6849970edec8819b9c42dcd537e81d370e37251b70a71f09', created_at=1749653263.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4o-2024-08-06', object='response', output=[ParsedResponseOutputMessage[CalendarEvent](id='msg_6849970fca60819b93866601af6b77850e37251b70a71f09', content=[ParsedResponseOutputText[CalendarEvent](annotations=[], text='{"name":"Science Fair","date":"Friday","participants":["Alice","Bob"]}', type='output_text', parsed=CalendarEvent(name='Science Fair', date='Friday', participants=['Alice', 'Bob']))], role='assistant', status='completed', type='message')], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[], top_p=1.0, max_output_tokens=None, previous_response_id=None, reasoning=Reasoning(effort=None, generate_summary=None, summary=None), service_tier='default', status='completed', text=ResponseTextConfig(format=ResponseFormatTextJSONSchemaConfig(name='CalendarEvent', schema_={'p

## 1. 체크리스트 생성 위젯

체크리스트 생성 기능을 위한 위젯 인터페이스를 생성합니다.

In [6]:
# 체크리스트 생성 매니저 생성
checklist_manager = ChecklistCreationManager()

# 위젯 표시
checklist_manager.display_all()

#### 예시
학년 : 1학년 \
과목 이름 : 국어 \
수행평가 제목 : 비혼주의자에 대한 본인의 의견\
수행평가 유형 : 찬성반대\
수행평가 설명 : 여러분은 결혼을 꼭 해야 한다고 생각하시나요? 아니면 결혼은 선택의 문제라고 생각하시나요? 비혼주의에 대해 찬성하는 입장 반대를 선택하여 자신의 의견을 작성해주세요. 그리고 그렇게 생각한 이유에 대해서 작성해주세요.

## 2. 과제 평가 위젯

학생 과제 평가 기능을 위한 위젯 인터페이스를 생성합니다.

#### 예시
학년 : 1학년 \
과목 이름 : 국어 \
수행평가 제목 : 비혼주의자에 대한 본인의 의견\
수행평가 유형 : 찬성반대\
수행평가 설명 : 여러분은 결혼을 꼭 해야 한다고 생각하시나요? 아니면 결혼은 선택의 문제라고 생각하시나요? 비혼주의에 대해 찬성하는 입장 반대를 선택하여 자신의 의견을 작성해주세요. 그리고 그렇게 생각한 이유에 대해서 작성해주세요.

In [3]:
# 과제 평가 매니저 생성
evaluation_manager = AssignmentEvaluationManager()

# 위젯 표시
evaluation_manager.display_all()

## 3. 단순 LLM API 콜 위젯
이 위젯은 단순한 LLM API 호출을 위한 것입니다.

In [4]:
# Create and display the widget manager
manager = LlmCallManager()
manager.display_all()

# AI Hub 데이터 자동 평가

In [5]:
import pandas as pd
from widgets_core import DEFAULT_SYSTEM_INSTRUCTION_EVALUATION, DEFAULT_PROMPT_EVALUATION, EVALUATION_SCHEMA
from llm_api_client import LLMAPIClient
import json
import re
import logging
from datetime import datetime
from tqdm.notebook import tqdm
from json_repair import repair_json

client = LLMAPIClient(log_level=logging.INFO)

In [6]:
df_aihub = pd.read_csv('C:/Users/yuhan/Desktop/CSD_18기/베어러블/AI_Hub/에세이_글_평가_데이터_전처리완.csv')

In [7]:
df_aihub

,student_grade,essay_id,essay_type,essay_main_subject,essay_prompt,processed_text,essay_score_org,essay_score_cont,essay_score_exp
0,고등_1학년,ESSAY_33982,글짓기,소중함을 잃었던 경험,익숙함에 속아 소중함을 잊지 말자는 명언 (Do not get fooled by ...,익숙함에 속아 소중함을 잊지말자라는 명언은 매우 유명한 명언이다. 난 이 명언의 내...,"[[3, 2, 3, 3], [2, 2, 3, 3], [2, 2, 2, 3]]","[[3, 2, 0, 3], [3, 2, 0, 2], [3, 2, 0, 2]]","[[3, 3, 2], [2, 2, 2], [3, 3, 2]]"
1,고등_1학년,ESSAY_33985,글짓기,소중함을 잃었던 경험,익숙함에 속아 소중함을 잊지 말자는 명언 (Do not get fooled by ...,소중함이란 가치를 따질 수 없을 만큼 매우 귀중한 것을 말합니다. 저에게도 이러한 ...,"[[2, 3, 3, 3], [3, 3, 3, 3], [2, 2, 3, 3]]","[[3, 2, 0, 3], [3, 3, 0, 3], [3, 3, 0, 3]]","[[3, 3, 2], [2, 2, 2], [2, 3, 2]]"
2,고등_1학년,ESSAY_33986,글짓기,소중함을 잃었던 경험,익숙함에 속아 소중함을 잊지 말자는 명언 (Do not get fooled by ...,"""익숙함에 속아 소중함을 잊지 말자"" 모든 사람들이 아는 말이다. 그만큼 모두가 그...","[[2, 3, 3, 2], [3, 3, 3, 2], [2, 2, 2, 3]]","[[3, 2, 0, 3], [3, 2, 0, 3], [3, 2, 0, 2]]","[[3, 3, 2], [3, 3, 3], [2, 2, 2]]"
3,고등_1학년,ESSAY_33988,글짓기,소중함을 잃었던 경험,익숙함에 속아 소중함을 잊지 말자는 명언 (Do not get fooled by ...,나는 가까운 사람에 대한 소중함에 대하여 알지 못했다. 그 일이 있기 전까지 말이다...,"[[3, 3, 3, 3], [2, 2, 3, 3], [2, 2, 2, 3]]","[[3, 3, 0, 3], [3, 3, 0, 2], [2, 2, 0, 2]]","[[3, 3, 3], [3, 3, 2], [2, 3, 2]]"
4,고등_1학년,ESSAY_33989,글짓기,소중함을 잃었던 경험,익숙함에 속아 소중함을 잊지 말자는 명언 (Do not get fooled by ...,나는 태어났을 때부터 할머니와 함께 살아왔는데 할머니는 하루도 빠짐없이 나에게 아침...,"[[2, 2, 3, 3], [2, 3, 3, 2], [3, 3, 3, 3]]","[[3, 2, 0, 3], [3, 3, 0, 2], [3, 2, 0, 3]]","[[3, 3, 3], [3, 3, 3], [3, 3, 3]]"
...,...,...,...,...,...,...,...,...,...
9454,고등_2학년,ESSAY_86639,찬성반대,익명성에 대한 본인의 의견,보편적으로 인터넷에서 사용자는 자기의 본이름을 제외한 ID나 별명을 사용합니다. ...,얼마전에 대선 후보인 윤석열 국민의힘이 장애인을 만나 정책을 얘을 하면서 잘못된 벌...,"[[1, 2, 1, 0], [1, 1, 0, 0], [2, 2, 2, 0]]","[[0, 0, 0, 0], [0, 0, 0, 0], [0, 1, 0, 0]]","[[2, 2, 1], [2, 2, 2], [2, 2, 1]]"
9455,고등_2학년,ESSAY_86640,찬성반대,익명성에 대한 본인의 의견,보편적으로 인터넷에서 사용자는 자기의 본이름을 제외한 ID나 별명을 사용합니다. ...,얼마전에 장애인 수영선수가 코로나백신 접종으로 인해 사망한 경우가 있다 사람들은 장...,"[[0, 1, 1, 0], [1, 2, 1, 0], [1, 1, 1, 0]]","[[0, 0, 0, 0], [0, 1, 0, 1], [0, 0, 0, 1]]","[[2, 2, 1], [2, 2, 2], [3, 3, 2]]"
9456,고등_2학년,ESSAY_86641,찬성반대,익명성에 대한 본인의 의견,보편적으로 인터넷에서 사용자는 자기의 본이름을 제외한 ID나 별명을 사용합니다. ...,옛날에 인기 있었던 영화 박하사탕을 보게 되었다 거기서 보면 문소리가 장애인으로 나...,"[[1, 1, 1, 0], [2, 2, 1, 0], [2, 2, 2, 0]]","[[0, 0, 0, 0], [0, 1, 0, 1], [0, 0, 0, 1]]","[[2, 2, 1], [2, 2, 2], [2, 2, 2]]"
9457,고등_2학년,ESSAY_86642,찬성반대,익명성에 대한 본인의 의견,보편적으로 인터넷에서 사용자는 자기의 본이름을 제외한 ID나 별명을 사용합니다. ...,"정신장애인의 경우 비정상적인 인간관계, 사회적 규범의 무시, 타인에 대한 의존 등 ...","[[0, 1, 1, 0], [1, 1, 1, 0], [1, 1, 1, 0]]","[[0, 0, 0, 0], [0, 1, 0, 1], [0, 0, 0, 1]]","[[2, 2, 1], [2, 2, 2], [3, 3, 2]]"


In [8]:
def create_input_dict(assignment_id_list, df, kwargs_dict):
    """
    주어진 과제 ID 리스트에 대한 입력 딕셔너리를 생성합니다.
    """
    
    # 과제 ID 리스트가 비어있는 경우
    if not assignment_id_list:
        print("Assignment ID list is empty.")
        return None

    # 과제 ID 리스트에 있는 과제만 추출
    selected_df = df[df['essay_id'].isin(assignment_id_list)]

    # 필요한 열 선택
    required_columns = ['essay_id', 'student_grade', 'essay_main_subject',
                        'essay_type', 'essay_prompt', 'processed_text']
    selected_df = selected_df[required_columns]
    
    # 열 이름 변경
    new_column_names = ['essay_id', 'student_grade', 'title', 'assessment_type', 'description', 'submission']
    selected_df.rename(columns=dict(zip(required_columns, new_column_names)), inplace=True)

    # 추가 정보 추가
    for key, value in kwargs_dict.items():
        selected_df[key] = value

    # essay_id를 인덱스로 설정하고 딕셔너리로 변환
    input_dict = selected_df.set_index('essay_id').T.to_dict()

    return input_dict

In [9]:
# Choose 5 random assignment IDs per assessment type
assignment_ids = []
for assessment_type in ['주장', '찬성반대', '대안제시']:
    sample_ids = df_aihub[df_aihub['essay_type'] == assessment_type]['essay_id'].sample(n=1, random_state=33).tolist()
    assignment_ids.extend(sample_ids)

input_dict = create_input_dict(assignment_ids, df_aihub, {'subject': '국어'})
input_dict.keys()

dict_keys(['ESSAY_78504', 'ESSAY_83769', 'ESSAY_74077'])

## UI에서 설정값 추출

In [10]:
def format_template(template, values, evaluation_criteria):
    """Format a template with the given values
    
    Args:
        template: The template string with [placeholders]
        values: Dictionary of placeholder values
        
    Returns:
        Formatted template with placeholders replaced
    """
    formatted = template
    replacements = {
        '학년': values.get('student_grade', ''),
        '과목': values.get('subject', ''),
        '수행평가 제목': values.get('title', ''),
        '수행평가 유형': values.get('assessment_type', ''),
        '수행평가 설명': values.get('description', ''),
        '평가 기준': json.dumps(evaluation_criteria, ensure_ascii=False, indent=2),
        '학생 제출물': values.get('submission', '')
    }
    
    for placeholder, value in replacements.items():
        if placeholder in formatted:
            formatted = formatted.replace(f"[{placeholder}]", value)
        
    return formatted

In [25]:
def clean_claude_json(response_text):
    """
    Custom function to clean and repair Claude's JSON responses
    that have specific formatting issues
    """
    # Step 1: Basic cleanup
    # Remove code block markers if present
    if "```json" in response_text:
        response_text = re.search(r'```json\s*(.*?)\s*```', response_text, re.DOTALL).group(1)
    
    # Step 2: Handle Claude's special evidence formatting
    # Find evidence arrays with mixed content types or text that includes other JSON markers
    evidence_pattern = re.compile(r'"evidence"\s*:\s*\[\s*(.*?)\s*\]', re.DOTALL)
    
    def fix_evidence_array(match):
        content = match.group(1).strip()
        
        # Check if it has the problematic format with unquoted text and commas
        if '"' in content and ('",' in content or '", "' in content):
            # It's probably a normal array of strings, but might have unquoted text at the end
            items = []
            # Split by commas but not inside quotes
            in_quotes = False
            current_item = ""
            
            for char in content:
                if char == '"' and (not current_item or current_item[-1] != '\\'):
                    in_quotes = not in_quotes
                
                if char == ',' and not in_quotes:
                    items.append(current_item.strip())
                    current_item = ""
                else:
                    current_item += char
            
            if current_item:
                items.append(current_item.strip())
            
            # Clean each item
            cleaned_items = []
            for item in items:
                item = item.strip()
                # If it's already quoted, keep it as is
                if item.startswith('"') and item.endswith('"'):
                    cleaned_items.append(item)
                # If it has unquoted text after quotes, extract and clean
                elif '"' in item:
                    # Try to find the quoted part
                    quoted_match = re.search(r'(".*?")(.*)', item)
                    if quoted_match:
                        quoted_part = quoted_match.group(1)
                        unquoted_part = quoted_match.group(2).strip()
                        # Only keep the quoted part if the unquoted part is just a comment
                        if unquoted_part.startswith('등') or not unquoted_part:
                            cleaned_items.append(quoted_part)
                        else:
                            # Try to reconstruct as a proper quoted string
                            cleaned_items.append(f'"{unquoted_part}"')
                else:
                    # It's completely unquoted, quote it
                    cleaned_items.append(f'"{item}"')
            
            return f'"evidence": [{", ".join(cleaned_items)}]'
        else:
            # It might be a complex structure, return as is
            return f'"evidence": [{content}]'
    
    # Replace all evidence arrays with cleaned versions
    response_text = evidence_pattern.sub(fix_evidence_array, response_text)
    
    # Step 3: Fix nested JSON objects that should be strings
    # Look for text fragments that appear to be JSON but should be strings
    nested_json_pattern = re.compile(r':\s*(\{(?:[^{}]|(?:\{[^{}]*\}))*\})')
    
    def fix_nested_json(match):
        json_obj = match.group(1)
        # Check if this is really a nested JSON or should be a string
        if len(json_obj) < 100:  # Short objects are probably legitimate
            return f": {json_obj}"
        else:
            # Long nested objects might be malformed - convert to string
            escaped_json_obj = json_obj.replace('"', '\\"')
            return f': "{escaped_json_obj}"'
    
    # Only apply this if we suspect issues
    if response_text.count('{') > response_text.count('}') + 3:
        response_text = nested_json_pattern.sub(fix_nested_json, response_text)
    
    # Step 4: Fix claude-specific structural issues
    # Replace all instances of 5+ consecutive newlines/spaces with proper JSON formatting
    response_text = re.sub(r'[\n\s]{5,}', ' ', response_text)
    
    # Step 5: Last resort - try to extract valid JSON from the mess
    try:
        return json.loads(response_text)
    except json.JSONDecodeError as e:
        print(f"Claude JSON cleaning failed at position {e.pos}: {response_text[max(0, e.pos-30):e.pos+30]}")
        
        # Try using the repair_json function as a fallback
        try:
            return json.loads(repair_json(response_text))
        except:
            # If all else fails, return an error object
            return {"error": "Failed to parse Claude JSON response"}

In [27]:
def automate_evaluation(input_dict, evaluation_schema, system_instruction_template, prompt_template, evaluation_manager=None):
    """
    주어진 입력 딕셔너리와 평가 기준을 사용하여 자동으로 평가를 수행합니다.
    Args:
        input_dict: 과제 정보가 포함된 딕셔너리
        evaluation_schema: 평가 기준
        system_instruction_template: 시스템 지침 템플릿
        prompt_template: 프롬프트 템플릿
        evaluation_manager: 평가 매니저 (선택적)
    Returns:
        평가 결과
    """
    result_dict = input_dict.copy()

    # 평가 기준 로드
    evaluation_criteria = None
    if evaluation_manager is not None:
        checklist_value = evaluation_manager.checklist_component.select_checklist_widget.value
        checklist_path = f'./checklists/{checklist_value}.json'
        evaluation_criteria = evaluation_manager.checklist_component.load_checklist(checklist_path)

    # 모델별 처리 정보
    model_configs = [
        {
            "name": "gemini",
            "api_func": lambda **kwargs: client.process_pdf_with_gemini(**kwargs),
            "response_getter": lambda resp: getattr(resp, "text", None),
            "model_name": lambda em: em.model_component.gemini_model_selection.value if em else 'gemini-2.0-flash',
            "params": lambda em: (
                em.model_component.gemini_params['max_tokens'].value if em else 4096,
                em.model_component.gemini_params['temperature'].value if em else 0.2
            )
        },
        {
            "name": "claude",
            "api_func": lambda **kwargs: client.process_pdf_with_claude(**kwargs),
            "response_getter": lambda resp: getattr(resp.content[0], "text", None) if resp and hasattr(resp, "content") else None,
            "model_name": lambda em: em.model_component.claude_model_selection.value if em else 'claude-3-7-sonnet-20250219',
            "params": lambda em: (
                em.model_component.claude_params['max_tokens'].value if em else 4096,
                em.model_component.claude_params['temperature'].value if em else 0.2
            )
        },
        {
            "name": "openai",
            "api_func": lambda **kwargs: client.process_pdf_with_openai(**kwargs),
            "response_getter": lambda resp: resp.choices[0].message.content if resp and hasattr(resp, "choices") else None,
            "model_name": lambda em: em.model_component.openai_model_selection.value if em else 'gpt-4.1',
            "params": lambda em: (
                em.model_component.openai_params['max_tokens'].value if em else 4096,
                em.model_component.openai_params['temperature'].value if em else 0.2
            )
        }
    ]

    for submission_id, value in tqdm(result_dict.items(), desc="Processing submissions"):
        # 시스템 지침 및 프롬프트 생성
        try:
            system_instruction = format_template(system_instruction_template, value, evaluation_criteria)
            prompt = format_template(prompt_template, value, evaluation_criteria)
        except Exception as e:
            print(f"Error formatting template for {submission_id}: {e}")
            continue

        for model in model_configs:
            model_name = model["model_name"](evaluation_manager)
            max_tokens, temperature = model["params"](evaluation_manager)
            api_func = model["api_func"]
            response_getter = model["response_getter"]

            # API 호출
            try:
                response = api_func(
                    file_path=None,
                    prompt=prompt,
                    model_name=model_name,
                    schema=evaluation_schema,
                    system_instruction=system_instruction,
                    max_tokens=max_tokens,
                    temperature=temperature
                )
            except Exception as e:
                print(f"Error processing with {model['name'].capitalize()}: {e}")
                response = None

            # 응답에서 텍스트 추출 + json 파싱 + 딕셔너리에 저장
            try:
                response_text = response_getter(response)
                if response_text and "```json" in response_text:
                    response_text = re.search(r'```json\s*(.*?)\s*```', response_text, re.DOTALL).group(1)
            except Exception as e:
                print(f"Error extracting {model['name'].capitalize()} response: {e}")
                response_text = "{'error': 'No JSON response found.'}"

            try:
                response_json = json.loads(repair_json(response_text))
            except Exception as e:
                print(f"Error parsing {model['name'].capitalize()} JSON: {e}")
                response_json = {"error": "Invalid JSON"}

            result_dict[submission_id][f"{model['name']}_model"] = model_name
            result_dict[submission_id][f"{model['name']}_response"] = response_json

            # 응답 저장
            try:
                if response:
                    print(f"Saving response for {submission_id} with {model['name']} model: {model_name}")
                    file_path = f"./auto_evaluation_results/{submission_id}_responses.txt"
                    with open(file_path, 'a', encoding='utf-8') as f:
                        f.write(f"Model: {model['name']} | {model_name}\n")
                        f.write(str(response))
                        f.write("\n\n")
            except Exception as e:
                print(f"Error saving responses to file: {e}")

    return result_dict

In [26]:
result_dict = automate_evaluation(
    input_dict,
    EVALUATION_SCHEMA,
    DEFAULT_SYSTEM_INSTRUCTION_EVALUATION,
    DEFAULT_PROMPT_EVALUATION,
    evaluation_manager
)

Processing submissions:   0%|          | 0/3 [00:00<?, ?it/s]

Saving response for ESSAY_78504 with model gemini
Saving response for ESSAY_78504 with model claude
Saving response for ESSAY_78504 with model openai
Saving response for ESSAY_83769 with model gemini
Saving response for ESSAY_83769 with model claude
Saving response for ESSAY_83769 with model openai
Saving response for ESSAY_74077 with model gemini
Saving response for ESSAY_74077 with model claude
Saving response for ESSAY_74077 with model openai


In [28]:
result_dict

{'ESSAY_78504': {'student_grade': '고등_1학년',
  'title': '남녀 갈등 해결 방안',
  'assessment_type': '대안제시',
  'description': ' 남녀 갈등은 최근 우리 사회에서 가장 뜨거운 이슈 중 하나입니다. 이러한 갈등은 아직 간극을 좁히지 못하고 현재 사회적인 문제로 자리 잡았습니다.\n\n 군 가산점 문제, 일자리 문제 등 여러 가지 비교 자료들을 토대로 토론을 하고 이를 통해 갈등을 해결하고자 하는 노력이 이루어지고 있습니다.\n\n 과거에는 성차별 심했던 시기가 있었습니다. 여자와 남자를 구별 지어 일을 주는 경우도 많았습니다. 하지만, 현재는 여자와 남자라는 성별을 떠나서 나이, 국적을 불문하고 모두에게 평등한 기회를 주기 위해 노력하고 있습니다.\n\n 이렇게 모두의 노력에도 불구하고 아직 해결되지 않았거나 해결될 기미가 보이지 않는 양성평등 문제가 있을까요?\n\n 아주 작은 사소한 일이라도 좋습니다. 여러분이 생각하는 우리 주변의 양성평등 문제를 하나 작성해보고 어떻게 해결할 수 있을지 혹은 무엇이 바뀌어야 해결될 것 같은지 본인의 생각을 설명해주세요. ',
  'submission': '아까 2번 문장(비혼주의자)에 대한 글에서 말씀했었던 것처럼 남녀 갈등에서는 1순위가 대부분 성격차이라고 했는데 사실은 남녀 갈등에서 여러 가지의 갈등을 보여 줄 수 있습니다. 사회에서도 남자는 군대를 가야하고 여자는 안 가는 고정관념과 남자는 힘이 있어 일자리를 제공해주고 여자는 연약해서 일자리를 제공할 수 없는 고정관념 등등 다양한 고정관념들을 볼 수 있습니다. 하지만 남자보다 여자가 더 유리할 수 밖에 없는 이유는 이러한 고정관념들 때문에 발생합니다. 남자는 상처가 나면 영광의 상처라고 생각하지만 여자가 상처가 나면 평생 지워지지 않은 흉터가 생기는 법처럼 남자는 항상 여자를 보호를 해야하는 고정관념이 살면서 가장 인상이 깊은 고정관념인 것 같습니다. 하지만 과거에서부터 남자만 군대 가는

In [29]:
# Save the result dictionary to a JSON file
output_file = './auto_evaluation_results/auto_evaluation_results_light_models.json'
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(result_dict, f, ensure_ascii=False, indent=2)

In [33]:
import numpy as np

# Load the JSON data
with open('./auto_evaluation_results/auto_evaluation_results_light_models.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Initialize lists to store our structured data
rows = []

# Process each essay
for essay_id, essay_data in data.items():
    # Basic essay information
    basic_info = {
        'Essay ID': essay_id,
        'Grade': essay_data['student_grade'],
        'Title': essay_data['title'],
        'Subject': essay_data['subject'],
        'Assessment Type': essay_data['assessment_type']
    }
    
    # Extract model names and responses
    models = {
        'Gemini': essay_data.get('gemini_model', 'N/A'),
        'Claude': essay_data.get('claude_model', 'N/A'),
        'OpenAI': essay_data.get('openai_model', 'N/A')
    }
    
    # Extract scores for each model and category
    scores = {}
    for model_name in ['gemini', 'claude', 'openai']:
        if f'{model_name}_response' in essay_data:
            response = essay_data[f'{model_name}_response']
            if 'evaluation' in response:
                for category in response['evaluation']:
                    cat_name = category['category']
                    for subcategory in category['subcategories']:
                        subcat_name = subcategory['name']
                        score_key = f"{model_name.capitalize()} - {cat_name} - {subcat_name}"
                        scores[score_key] = subcategory['score']
    
    # Combine all information
    row_data = {**basic_info, **models, **scores}
    rows.append(row_data)

# Create DataFrame
df = pd.DataFrame(rows)

# Organize columns
main_cols = ['Essay ID', 'Grade', 'Title', 'Subject', 'Assessment Type', 'Gemini', 'Claude', 'OpenAI']
score_cols = [col for col in df.columns if col not in main_cols]

# Sort score columns by model, category, and subcategory
sorted_score_cols = sorted(score_cols)

# Final column order
final_cols = main_cols + sorted_score_cols

# Reorder columns
df = df[final_cols]

# Create a styled dataframe for better visualization
def highlight_scores(s):
    if pd.api.types.is_numeric_dtype(s):
        return ['background-color: #d4f1f9' if v == 3 else 
                'background-color: #ffeb99' if v == 2 else
                'background-color: #ffb3b3' if v == 1 else
                'background-color: #ff8080' if v == 0 else '' for v in s]
    return [''] * len(s)

# Function to display the DataFrame in a readable format
def display_dataframe(df):
    # Display basic essay info
    print("ESSAY INFORMATION:")
    basic_info_df = df[main_cols].copy()
    print(basic_info_df.to_string(index=False))
    print("\n" + "-"*100 + "\n")
    
    # Display scores by model and category
    # Group score columns by model and category
    score_data = {}
    for col in sorted_score_cols:
        parts = col.split(' - ')
        if len(parts) >= 3:
            model, category, subcategory = parts[0], parts[1], ' - '.join(parts[2:])
            if model not in score_data:
                score_data[model] = {}
            if category not in score_data[model]:
                score_data[model][category] = {}
            for i, row in df.iterrows():
                essay_id_val = row['Essay ID']
                if essay_id_val not in score_data[model][category]:
                    score_data[model][category][essay_id_val] = {}
                score_data[model][category][essay_id_val][subcategory] = row.get(col, np.nan)
    
    # Display scores for each model
    for model in ['Gemini', 'Claude', 'OpenAI']:
        if model in score_data:
            print(f"\n{model.upper()} MODEL SCORES:")
            for category in ['표현', '구성', '내용']:
                if category in score_data[model]:
                    print(f"\n{category} Category:")
                    category_data = []
                    for essay_id_val in df['Essay ID']:
                        if essay_id_val in score_data[model][category]:
                            row_data = {'Essay ID': essay_id_val, **score_data[model][category][essay_id_val]}
                            category_data.append(row_data)
                    
                    if category_data:
                        cat_df = pd.DataFrame(category_data)
                        print(cat_df.to_string(index=False))
                        cat_df = pd.DataFrame(category_data)
                        print(cat_df.to_string(index=False))

# Display the dataframe
display_dataframe(df)

# Save to CSV for further analysis if needed
# df.to_csv('evaluation_results_table.csv', index=False)
# print("\nTable saved to 'evaluation_results_table.csv'")

ESSAY INFORMATION:
   Essay ID  Grade                Title Subject Assessment Type                Gemini                    Claude       OpenAI
ESSAY_78504 고등_1학년          남녀 갈등 해결 방안      국어            대안제시 gemini-2.0-flash-lite claude-3-5-haiku-20241022 gpt-4.1-mini
ESSAY_83769 고등_2학년 지적 재산권에 대한 본인의 생각 작성      국어              주장 gemini-2.0-flash-lite claude-3-5-haiku-20241022 gpt-4.1-mini
ESSAY_74077 고등_2학년       익명성에 대한 본인의 의견      국어            찬성반대 gemini-2.0-flash-lite claude-3-5-haiku-20241022 gpt-4.1-mini

----------------------------------------------------------------------------------------------------


GEMINI MODEL SCORES:

표현 Category:
   Essay ID  단어 사용의 적절성  문법의 정확성  문장 표현의 적절성
ESSAY_78504           2        2           1
ESSAY_83769           2        2           1
ESSAY_74077           2        2           2
   Essay ID  단어 사용의 적절성  문법의 정확성  문장 표현의 적절성
ESSAY_78504           2        2           1
ESSAY_83769           2        2           1
ESSAY_74077           2    

In [31]:
# Load the JSON data
with open('./auto_evaluation_results/auto_evaluation_results_light_models.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Create a list to store model comparison data
comparison_data = []

# Process each essay
for essay_id, essay_data in data.items():
    # Create category-based scoring dictionaries for each model
    model_scores = {
        'Gemini': {'표현': {}, '구성': {}, '내용': {}},
        'Claude': {'표현': {}, '구성': {}, '내용': {}},
        'OpenAI': {'표현': {}, '구성': {}, '내용': {}}
    }
    
    # Extract scores for each model
    for model_key, model_name in [('gemini', 'Gemini'), ('claude', 'Claude'), ('openai', 'OpenAI')]:
        if f'{model_key}_response' in essay_data and 'evaluation' in essay_data[f'{model_key}_response']:
            for category in essay_data[f'{model_key}_response']['evaluation']:
                cat_name = category['category']
                for subcategory in category['subcategories']:
                    subcat_name = subcategory['name']
                    model_scores[model_name][cat_name][subcat_name] = subcategory['score']
    
    # Create a row for this essay
    row = {
        'Essay ID': essay_id,
        'Title': essay_data['title'],
        'Grade': essay_data['student_grade'],
        'Subject': essay_data['subject'],
        'Assessment Type': essay_data['assessment_type']
    }
    
    # Add model scores to the row
    for model_name in ['Gemini', 'Claude', 'OpenAI']:
        # Add model version
        row[f'{model_name} Model'] = essay_data.get(f'{model_name.lower()}_model', 'N/A')
        
        # Add category scores
        for category in ['표현', '구성', '내용']:
            for subcategory, score in model_scores[model_name][category].items():
                row[f'{model_name} - {category} - {subcategory}'] = score
    
    comparison_data.append(row)

# Create DataFrame
comparison_df = pd.DataFrame(comparison_data)

# Style the DataFrame for better HTML visualization
def highlight_scores(val):
    if pd.isna(val) or not isinstance(val, (int, float)):
        return ''
    
    score_colors = {
        0: 'background-color: #ff8080',  # Red
        1: 'background-color: #ffcccc',  # Light red
        2: 'background-color: #ffffcc',  # Light yellow
        3: 'background-color: #ccffcc'   # Light green
    }
    
    return score_colors.get(val, '')

# Apply styling
styled_df = comparison_df.style.applymap(highlight_scores, subset=[col for col in comparison_df.columns if any(model in col for model in ['Gemini - ', 'Claude - ', 'OpenAI - '])])

# Output to HTML
html_table = styled_df.to_html()

# Save the HTML
with open('model_comparison_table.html', 'w', encoding='utf-8') as f:
    f.write("""
    <!DOCTYPE html>
    <html>
    <head>
        <title>LLM Evaluation Comparison</title>
        <style>
            body {
                font-family: Arial, sans-serif;
                margin: 20px;
            }
            table {
                border-collapse: collapse;
                width: 100%;
                margin-top: 20px;
            }
            th, td {
                border: 1px solid #ddd;
                padding: 8px;
                text-align: left;
            }
            th {
                background-color: #f2f2f2;
                position: sticky;
                top: 0;
            }
            tr:hover {background-color: #f5f5f5;}
            .score-header {
                background-color: #e6f3ff;
            }
        </style>
    </head>
    <body>
        <h1>LLM Evaluation Comparison</h1>
    """ + html_table + """
    </body>
    </html>
    """)

print("Comparison table saved as 'model_comparison_table.html'")

ImportError: Missing optional dependency 'Jinja2'. DataFrame.style requires jinja2. Use pip or conda to install Jinja2.